<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/1.%20Categorical_Feature_Encoding_Challenge_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
# !kaggle competitions list

In [ ]:
!kaggle competitions download -c cat-in-the-dat

 85% 18.0M/21.2M [00:01<00:00, 25.9MB/s]
100% 21.2M/21.2M [00:01<00:00, 18.9MB/s]


In [ ]:
!ls

cat-in-the-dat.zip  kaggle.json  sample_data


In [ ]:
!unzip cat-in-the-dat.zip

Archive:  cat-in-the-dat.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# **1. 데이터 준비**

데이터를 준비하겠습니다.

In [ ]:
import numpy as np
import pandas as pd

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 50)
pd.set_option('display.max_rows', 50)

# 데이터를 읽습니다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# **2. 데이터 확인**

## **2.1. 데이터 개요**

In [ ]:
train.head(3)

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      300000 non-null  int64 
 1   bin_0   300000 non-null  int64 
 2   bin_1   300000 non-null  int64 
 3   bin_2   300000 non-null  int64 
 4   bin_3   300000 non-null  object
 5   bin_4   300000 non-null  object
 6   nom_0   300000 non-null  object
 7   nom_1   300000 non-null  object
 8   nom_2   300000 non-null  object
 9   nom_3   300000 non-null  object
 10  nom_4   300000 non-null  object
 11  nom_5   300000 non-null  object
 12  nom_6   300000 non-null  object
 13  nom_7   300000 non-null  object
 14  nom_8   300000 non-null  object
 15  nom_9   300000 non-null  object
 16  ord_0   300000 non-null  int64 
 17  ord_1   300000 non-null  object
 18  ord_2   300000 non-null  object
 19  ord_3   300000 non-null  object
 20  ord_4   300000 non-null  object
 21  ord_5   300000 non-null  object
 

일견 결측치는 없어 보입니다. 

In [ ]:
train.describe()

,id,bin_0,bin_1,bin_2,ord_0,day,month,target
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.00000
mean,149999.500000,0.127503,0.256607,0.383223,1.479567,3.007817,5.771113,0.30588
std,86602.684716,0.333537,0.436761,0.486173,0.712524,1.817949,3.843519,0.46078
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.00000
25%,74999.750000,0.000000,0.000000,0.000000,1.000000,2.000000,2.000000,0.00000
50%,149999.500000,0.000000,0.000000,0.000000,1.000000,3.000000,4.000000,0.00000
75%,224999.250000,0.000000,1.000000,1.000000,2.000000,4.000000,9.000000,1.00000
max,299999.000000,1.000000,1.000000,1.000000,3.000000,7.000000,12.000000,1.00000


연속형 피처가 없기 때문에 `describe()` 메서드로 얻을 만한 유용한 정보는 없습니다. 이 데이터의 모든 피처는 범주형입니다.

In [ ]:
test.head(3)

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,0870b0a5d,9ceb19dd6,530f8ecc3,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
1,300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,a5c276589,1ad744242,12e6161c9,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
2,300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,568550f04,1fe17a1fd,27d6df03f,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      200000 non-null  int64 
 1   bin_0   200000 non-null  int64 
 2   bin_1   200000 non-null  int64 
 3   bin_2   200000 non-null  int64 
 4   bin_3   200000 non-null  object
 5   bin_4   200000 non-null  object
 6   nom_0   200000 non-null  object
 7   nom_1   200000 non-null  object
 8   nom_2   200000 non-null  object
 9   nom_3   200000 non-null  object
 10  nom_4   200000 non-null  object
 11  nom_5   200000 non-null  object
 12  nom_6   200000 non-null  object
 13  nom_7   200000 non-null  object
 14  nom_8   200000 non-null  object
 15  nom_9   200000 non-null  object
 16  ord_0   200000 non-null  int64 
 17  ord_1   200000 non-null  object
 18  ord_2   200000 non-null  object
 19  ord_3   200000 non-null  object
 20  ord_4   200000 non-null  object
 21  ord_5   200000 non-null  object
 

학습 데이터와 다른 점은 'target' 피처가 없는 것입니다. 이외 특이사항은 없어 보입니다.

## **2.2. 피처 개요**

데이터가 가진 피처들의 특징은 아래와 같습니다.

- 인위적으로 생성한 데이터이므로 피처 자체가 특별한 의미를 내포하지 않습니다.

- 피처명에 따라 피처 유형을 세분화합니다. 참고로 모든 피처는 범주형입니다. 연속형 피처는 존재하지 않습니다.

 - 'bin_num': 이진형 피처

 - 'nom_num': 명목형 피처

 - 'ord_num': 순서형 피처('ord_3', 'ord_4', 'ord_5'는 알파벳순)

 - 'day', 'month': 날짜 피처

'id' 피처는 레코드 순서를 나타내는 것으로 보이며, 실제로 예측에 어떠한 영향도 주지 않을 것으로 예상합니다. 고윳값을 확인해 보겠습니다.

In [ ]:
train['id'].nunique()

300000

데이터 크기와 동일하게 고윳값이 존재합니다. 'id' 피처는 인덱스로 지정하겠습니다. 물론 피처 자체를 삭제해도 좋습니다.

In [ ]:
train = train.set_index('id')
test = test.set_index('id')
submission = submission.set_index('id')

### **2.2.1. 피처 요약**

먼저 모든 피처를 요약하는 함수를 선언하여 피처가 가진 특징을 확인하겠습니다.

In [ ]:
def feature_table(df):
    # 데이터 크기를 출력합니다.    
    print(df.shape)
    # 피처 이름과 유형을 담은 데이터프레임을 생성합니다.   
    feature_description = pd.DataFrame(df.dtypes, columns=['type'])
    # 피처 이름 피처가 인덱스이므로 인덱스를 리셋하여 새 인덱스를 만듭니다.
    feature_description = feature_description.reset_index()
    # 피처 이름을 값으로 가진 피처명은 'index'입니다. 'feature'로 변환합니다.
    feature_description = feature_description.rename(columns={'index': 'feature'})
    # 각 피처가 가진 첫 번째 ~ 다섯 번째 값을 출력합니다.
    feature_description['first_value'] = df.loc[0].values
    feature_description['second_value'] = df.loc[1].values
    feature_description['third_value'] = df.loc[2].values
    feature_description['fourth_value'] = df.loc[3].values
    feature_description['fifth_value'] = df.loc[4].values
    # 결측치 개수를 산출합니다.
    feature_description['null_count'] = df.isnull().sum().values
    # 고윳값 개수를 산출합니다.
    feature_description['eigenvalue_count'] = df.nunique().values

    return feature_description

In [ ]:
feature_table(train)

(300000, 24)


,feature,type,first_value,second_value,third_value,fourth_value,fifth_value,null_count,eigenvalue_count
0,bin_0,int64,0,0,0,0,0,0,2
1,bin_1,int64,0,1,0,1,0,0,2
2,bin_2,int64,0,0,0,0,0,0,2
3,bin_3,object,T,T,F,F,F,0,2
4,bin_4,object,Y,Y,Y,Y,N,0,2
5,nom_0,object,Green,Green,Blue,Red,Red,0,3
6,nom_1,object,Triangle,Trapezoid,Trapezoid,Trapezoid,Trapezoid,0,6
7,nom_2,object,Snake,Hamster,Lion,Snake,Lion,0,6
8,nom_3,object,Finland,Russia,Russia,Canada,Canada,0,6
9,nom_4,object,Bassoon,Piano,Theremin,Oboe,Oboe,0,4


- 모든 피처에 결측치는 없습니다.

- 모든 피처가 예측에 유효할 것이므로 최대한 인코딩과 스케일링을 적용하여 예측에 활용해야 합니다.

- 'bin_3', 'bin_4' 피처는 0, 1이 아닌 (T, F), (Y, N)으로 구성된 이진형 피처입니다. 각각 1, 0으로 매핑하여 인코딩을 수행할 필요가 있습니다.

- 'nom_1' ~ 'nom_9' 피처는 고윳값이 적은 피처부터 매우 많은 피처까지 다양합니다. 원-핫 인코딩을 적용할 필요가 있습니다. 다만 매우 많은 피처가 생성되어 학습 시간이 오래 걸릴 수 있습니다.

- 'ord_0' ~ 'ord_5' 피처는 고윳값이 적은 피처부터 많은 피처까지 다양합니다. 순서형 피처에 인코딩을 적용하는 경우 순서가 가진 함의를 잃지 않도록 세심하게 접근해야 합니다. OrdinalEncoder를 적용할 만합니다.

- 'day', 'month' 피처는 정숫값으로 구성된 범주형 피처입니다. 그대로 사용해도 무방하지만 원-핫 인코딩을 적용하여 약간의 성능 향상이라도 도모해 볼 수 있을 것입니다. 

- 'target' 피처는 0, 1로 구성된 타깃 피처입니다. 



---

업데이트 중

---

